## We will implement a new forward index and save them in barrels

In [1]:
import docdata.protobufs.doc_file_pb2 as dpb # Importing docdata protobuf data
import forwardindex.protobufs.forwardindex_file_pb2 as fpb # Importing forwardindex protobuf data
from metadata.metadatafuncs import get_metadata, save_metadata # Getting the metadata
from wordlexicon.trie import Trie, TrieNode
import wordlexicon.wordlexicon as wl # Importing wordlexicon
import os # Importing os to get directory data of each os file
from forwardindex.parse_forward import parse # Getting each document element from here
from docdata import save_docs # Importing doc_data to save document data
from forwardindex.getjson import get_jsondata # Getting the json data
import time
import sys
import pickle
from forwardindex.parsed_items.parsed import get_parsed_set, set_parsed_set

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## We will implement both docdata and forwardindex here

In [2]:
os.listdir("forwardindex/JSONFILES/dir")

['thesun.json',
 'newsbusters.json',
 'usnews.json',
 'cnn.json',
 'yahoonews.json',
 'theindependent.json',
 'theguardianuk.json',
 'thenewyorktimes.json']

In [16]:
json_data = get_jsondata("thesun.json")

71109


In [17]:
json_data[:10]

[{'id': 'thesun--2022-01-01--Fearne Cotton and Jake Humphrey reveal ‘aggressive’ off-screen tension at CBBC',
  'date': '2022-01-01',
  'source': 'thesun',
  'title': 'Fearne Cotton and Jake Humphrey reveal ‘aggressive’ off-screen tension at CBBC',
  'content': "They claimed that , while families saw only smiley faces , tears flowed off screen as egos clashed .\nFearne , on the channel from 2001 before fronting Top of the Pops , said it left her “ very unhappy ” .\nFormula One host Jake , 43 , another children ’ s TV veteran , said : “ CBBC was probably the most competitive and aggressive and sort of catty place .\n“ Everyone was young .\nEverybody wanted to be the next big thing.\n” He asked Fearne , 40 , on a podcast : “ Have you @ @ @ @ @ @ @ many tears ?\n” She replied : “ I used to stay out of it .\nIt was too scary for me.\n” Jake , 43 , presented CBBC 's BAMZOOKi , Against All Odds , and Rule The School , as well as hosting its coverage of Fame Academy with now This Morning star

In [9]:
# set_parsed_set(set())
lexicon_height, totaldocs, forwardindexlimit = get_metadata()
# save_metadata(lexicon_height, 0, forwardindexlimit)
print(lexicon_height)
print(forwardindexlimit)

239077
5000


In [11]:
# This function will basically add the forward index from jsonfiles
def addFIndex(json_data, barrels):
    lexicon_height, totaldocs, forwardindexlimit = get_metadata()
    doc_id = totaldocs
    forwardindex = fpb.ForwardIndex()
    doc_data = dpb.DocData()
    forwardindex_path = "forwardindex/data/findex_{}.pb"
    docdata_path = "docdata/docdata_{}.pb"
    currset = get_parsed_set()
    start_time=time.time()
    count = 0
    for eacharticle in json_data:
        if eacharticle['url'] not in currset:
            currset.add(eacharticle['url'])
            forwardindex.docelement.append(parse(eacharticle, doc_id))
            doc_data.eachdoc.append(save_docs.parse(eacharticle))
            count+=1
            doc_id+=1
            if doc_id % forwardindexlimit == 0:
                end_time = time.time()
                print(doc_id, " - doc_id in", end_time-start_time, "seconds")
                file_no = doc_id/forwardindexlimit
                with open(forwardindex_path.format(file_no), "wb") as f:
                    f.write(forwardindex.SerializeToString())
                with open(docdata_path.format(file_no), "wb") as f:
                    f.write(doc_data.SerializeToString())
                forwardindex = fpb.ForwardIndex()
                doc_data = dpb.DocData()
            if count == barrels*forwardindexlimit:
                wl.UpdateLexicon()
                newlexsize = wl.GetLexiconSize()
                save_metadata(newlexsize, doc_id, forwardindexlimit)
                set_parsed_set(currset)
                print("Metadata saved with lexsize = ", newlexsize, "| total_docs = ", doc_id)
                return

In [14]:
addFIndex(json_data, 5)

Current size of set:  4194520  bytes
105000  - doc_id in 6.898204326629639 seconds
110000  - doc_id in 14.053057432174683 seconds
115000  - doc_id in 21.48855471611023 seconds
120000  - doc_id in 28.71401810646057 seconds
125000  - doc_id in 36.479591369628906 seconds
Metadata saved with lexsize =  341310 | total_docs =  125000


In [48]:
wl.GetLexiconSize()

250829

In [8]:
currset = get_parsed_set()

Current size of set:  4194520  bytes


In [9]:
len(list(currset))

115000

In [11]:
forwardindex = fpb.ForwardIndex()
with open("forwardindex/data/findex_1.0.pb", "rb") as f:
    filecontent = f.read()
forwardindex.ParseFromString(filecontent)

20786460

In [15]:
len(forwardindex.docelement)

5000